In [1]:
import sqlite3
con = sqlite3.connect('db')

In [2]:
import pandas as pd

In [3]:
sqlite3.sqlite_version

'3.39.4'

In [4]:
df = pd.read_csv('database/german_credit_augmented.csv')
df.head()

,age,sex,job,housing,saving_accounts,checking_account,credit_amount,duration,purpose,default,contract_dt,client_id
0,33,male,2,own,NaN,NaN,3074,9,radio/TV,0,2008-06-29 18:52:00,210
1,43,male,1,own,little,little,1344,12,car,0,2007-05-20 18:30:19,929
2,52,male,2,own,quite rich,NaN,936,9,education,0,2008-04-27 08:23:07,200
3,35,female,3,own,little,NaN,1393,11,car,0,2007-05-06 10:58:22,45
4,28,male,2,own,little,NaN,776,12,radio/TV,0,2007-07-21 13:22:14,358


In [5]:
df.to_sql('german_credit', con, index=False, if_exists='replace')

1000

In [6]:
def select(sql):
    return pd.read_sql(sql, con)

In [7]:
users = pd.DataFrame({'id':[1,2,3],'name':['gleb','jon snow','tyrion']})

In [8]:
users

,id,name
0,1,gleb
1,2,jon snow
2,3,tyrion


In [9]:
items = pd.DataFrame({'user_id':[1,3,3],'item_name':['hleb','gold','wine'],'value':[5,100,20]})

In [10]:
items

,user_id,item_name,value
0,1,hleb,5
1,3,gold,100
2,3,wine,20


In [11]:
users.to_sql('users',con,index=False,if_exists='replace')
items.to_sql('items',con,index=False,if_exists='replace')

3

In [12]:
sql = '''select t.*, i.item_name, i.value, i.user_id from users t
left join items i on t.id = i.user_id
'''

In [13]:
select(sql)

,id,name,item_name,value,user_id
0,1,gleb,hleb,5.0,1.0
1,2,jon snow,None,NaN,NaN
2,3,tyrion,gold,100.0,3.0
3,3,tyrion,wine,20.0,3.0


In [14]:
sql = '''select t.*, i.item_name from users t
join items i on t.id = i.user_id
'''

In [15]:
select(sql)

,id,name,item_name
0,1,gleb,hleb
1,3,tyrion,gold
2,3,tyrion,wine


In [16]:
sql = '''select t.*, i.item_name from users t
left join items i on t.id = i.user_id
where i.item_name is not null
'''

In [17]:
select(sql)

,id,name,item_name
0,1,gleb,hleb
1,3,tyrion,gold
2,3,tyrion,wine


In [18]:
users = pd.DataFrame({'id':[1,2,3],'name':['gleb','jon snow','tyrion'],
                      'victory':[2,10,1]})

In [19]:
users.to_sql('users',con,index=False,if_exists='replace')

3

In [20]:
sql = '''select t.*, i.item_name, i.value, i.user_id from users t
left join items i on t.id = i.user_id
'''

In [21]:
t = select(sql)

In [23]:
t

,id,name,victory,item_name,value,user_id
0,1,gleb,2,hleb,5.0,1.0
1,2,jon snow,10,None,NaN,NaN
2,3,tyrion,1,gold,100.0,3.0
3,3,tyrion,1,wine,20.0,3.0


In [24]:
t['victory'].sum()

14

In [25]:
sql = '''select sum(t.victory) from users t'''

In [26]:
select(sql)

,sum(t.victory)
0,13


In [27]:
sql = '''select t.id, t.name,
t.victory,
count(i.item_name) as item_cnt,
coalesce(sum(i.value),0) as value_sum

from users t
left join items i on t.id = i.user_id
group by t.id, t.name, t.victory
'''

In [28]:
select(sql)

,id,name,victory,item_cnt,value_sum
0,1,gleb,2,1,5
1,2,jon snow,10,0,0
2,3,tyrion,1,2,120


In [29]:
sql = '''
with items_agg as (

select t.user_id, count(t.item_name) as item_cnt,
sum(value) as value_sum from items t
group by t.user_id)

select t.id, t.name,
t.victory,

coalesce(i.item_cnt,0) as item_cnt,
coalesce(i.value_sum,0) as value_sum

from users t

left join items_agg i on t.id = i.user_id

'''

In [30]:
select(sql)

,id,name,victory,item_cnt,value_sum
0,1,gleb,2,1,5
1,2,jon snow,10,0,0
2,3,tyrion,1,2,120


In [31]:
"Никогда не используй right join!"

'Никогда не используй right join!'

In [32]:
users

,id,name,victory
0,1,gleb,2
1,2,jon snow,10
2,3,tyrion,1


In [33]:
items = pd.DataFrame({'user_id':[1,3,3,4],'item_name':['hleb','gold','wine','sword'],'value':[5,100,20,50]})

In [34]:
items

,user_id,item_name,value
0,1,hleb,5
1,3,gold,100
2,3,wine,20
3,4,sword,50


In [35]:
users.to_sql('users',con,index=False,if_exists='replace')
items.to_sql('items',con,index=False,if_exists='replace')

4

In [36]:
sql = '''select t.*, i.* from users t
left join items i on t.id = i.user_id'''

In [37]:
select(sql)

,id,name,victory,user_id,item_name,value
0,1,gleb,2,1.0,hleb,5.0
1,2,jon snow,10,NaN,None,NaN
2,3,tyrion,1,3.0,gold,100.0
3,3,tyrion,1,3.0,wine,20.0


In [38]:
"Inner join"

'Inner join'

In [39]:
sql = '''select * from german_credit t
limit 5'''

In [40]:
select(sql)

,age,sex,job,housing,saving_accounts,checking_account,credit_amount,duration,purpose,default,contract_dt,client_id
0,33,male,2,own,None,None,3074,9,radio/TV,0,2008-06-29 18:52:00,210
1,43,male,1,own,little,little,1344,12,car,0,2007-05-20 18:30:19,929
2,52,male,2,own,quite rich,None,936,9,education,0,2008-04-27 08:23:07,200
3,35,female,3,own,little,None,1393,11,car,0,2007-05-06 10:58:22,45
4,28,male,2,own,little,None,776,12,radio/TV,0,2007-07-21 13:22:14,358


In [41]:
clients = pd.DataFrame({'client_id':[200,45],'data':[1,2]})

In [42]:
clients.to_sql('clients_task_name',con,index=False,if_exists='replace')

2

In [43]:
sql = '''select t.*, ctn.data from german_credit t
join clients_task_name ctn on t.client_id = ctn.client_id
'''

In [44]:
select(sql)

,age,sex,job,housing,saving_accounts,checking_account,credit_amount,duration,purpose,default,contract_dt,client_id,data
0,52,male,2,own,quite rich,None,936,9,education,0,2008-04-27 08:23:07,200,1
1,35,female,3,own,little,None,1393,11,car,0,2007-05-06 10:58:22,45,2


In [45]:
sql = '''select 1 as user_id
union all
select 2 as user_id
union all
select 3 as user_id'''

In [46]:
select(sql)

,user_id
0,1
1,2
2,3


In [47]:
sql = '''select date('2021-03-01') as month
union all
select date('2021-04-01') as month
'''

In [48]:
select(sql)

,month
0,2021-03-01
1,2021-04-01


In [49]:
sql = '''
with users as (
select 1 as user_id
union all
select 2 as user_id
union all
select 3 as user_id),

month as

(select date('2021-03-01') as month
union all
select date('2021-04-01') as month)


select * from users t
join month m on 1=1

'''

In [50]:
select(sql)

,user_id,month
0,1,2021-03-01
1,2,2021-03-01
2,3,2021-03-01
3,1,2021-04-01
4,2,2021-04-01
5,3,2021-04-01
